### Lost feature code

Main differencce with `buid_tree.ipynb` is slightly different prompt and removing `Glucose` feature.

In [6]:
dataset_name = "diabetes"
results_path = f"tree_scores.{dataset_name}_lost_feature.json"

In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd
import openml
import smolagents
from huggingface_hub import login
import proxy_api_model
import prompting
import tree_agent
from sklearn.model_selection import train_test_split
from task import metric_func_by_task, get_task_variables, add_tabpfn_baseline
import dataset_descriptions

login(token=HF_TOKEN_HERE)
# V-- this uses a GPT-5 model over an API. Replace with https://smolagents.org/docs/agents-guided-tour/
model = proxy_api_model.ProxyAPIModel(
    api_key=ELIZA_TOKEN,
    api_base=API_ENDPOINT_HERE,  # <-- https://your/openai-like/api/v1/chat/completions
    api_key=API_TOKEN_HERE,  # <-- use your token
    max_new_tokens=1024 * 8,
    callback=lambda msg, **etc: print(  # print model thoughts before code
    re.sub(r'<code>.*?</code>', '<code omitted>', msg.content, flags=re.DOTALL))
)

# Load tabular benchmark
tabarena_version = "tabarena-v0.1"
benchmark_suite = openml.study.get_suite(tabarena_version)
task_ids = benchmark_suite.tasks
dataset_name_to_task_id = {}
for task_id in task_ids:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    n_samples = dataset.qualities["NumberOfInstances"]
    if n_samples < 2_500:
        dataset_name_to_task_id[dataset.name] = task_id
        print(dataset.name, int(n_samples), task_id)

dataset_path = os.path.join("data", dataset_name)
assert not dataset_path.endswith('/')

In [8]:
instructions = f"""
We forgot to collect `Glucose` data in the training set. This feature is actually very important for predicting diabetes, but you cannot see it in train. Your task is to create a decision tree that may rely on this feature in the test set, even though you have no Glucose data available during training.
""".strip()

In [ ]:
test_scores = []
for repeat_index in range(5):
    print("Beginning repeat", repeat_index)
    task = openml.tasks.get_task(dataset_name_to_task_id[dataset_name])
    data = get_task_variables(task, fold=0, repeat=repeat_index)
    task_type = data['task_type']
    X_train, X_val, y_train, y_val = train_test_split(
        data["X_train"], data["y_train"], test_size=0.2, random_state=42,
        stratify=data['y_train'] if data['task_type'] != 'regression' else None
    )
    X_test, y_test = data["X_test"], data["y_test"]

    X_train = X_train.drop("Glucose", axis=1)
    # X_test = X_test.drop("Glucose", axis=1)

    result = tree_agent.TreeAgent(model=model).run(
        task=f"""
Build the optimal decision tree for the '{dataset_name}' dataset.
You are given access to 4 data variables in your python environment:
 - X_train, X_val are pandas dataframes with named feature columns (see below) that may need preprocessing;
 - y_train, y_val are numpy arrays (1d) with targets, also described below;

Dataset description (use it to form hypotheses):
{dataset_descriptions.desc.get(dataset_name).format(num_samples=len(X_train), metric=prompting.metrics_by_task[task_type])}


Here's additional instructions you should follow:
{instructions}

Here's one way you could construct before you begin editing it manually:
{prompting.starter_snippets_by_task[task_type]}

Now begin: view the data variables, preprocess as necessary, train a baseline tree, then propose the first hypothesis and start improving.
Focus on drawing conclusions from data, looking at the tree (e.g. via print) and using your own intuition about the problem for manual tree edits.
Quality is more important than speed: take as many steps as you need to get the best tree.
""".strip(),
        additional_args=dict(
            X_train=X_train.copy(), y_train=y_train.copy(),
            X_val=X_val.copy(), y_val=y_val.copy(),
        )
    )

    y_pred_i = result['model'].predict(result['preprocess_features'](X_test.copy()))
    if task_type == 'multiclass':  # normalize for logloss
        y_pred_i = y_pred_i / y_pred_i.sum(axis=-1, keepdims=True)
    test_score = metric_func_by_task[task_type](y_test, y_pred_i)
    print(f"Test {prompting.metrics_by_task[task_type]} score #{repeat_index}: {test_score:.5f}")
    test_scores.append(test_score)
    with open(results_path, "w") as f:
        json.dump(test_scores, f)